# 0. Imports

In [ ]:
import os
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import h3
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio

import matplotlib.pyplot as plt
import pydeck as pdk

from shapely import Point
from gcsfs import GCSFileSystem

from research.analysis.transform.h3 import point_to_h3, raster_band_to_pandas_h3
from research.viz.plot import equal_color_intervals

# 1. Leemos nuestros datasets

In [ ]:
gcs_bucket = os.getenv("GCS_BUCKET_URI")
fs = GCSFileSystem()

In [ ]:
soja_gdf = gpd.read_parquet(
    f'{gcs_bucket}/spam_soy_bean_global_production_2010.parquet', 
    filesystem=fs
)

soja_gdf.head(3)

In [ ]:
aoi_df = pd.read_csv(f'{gcs_bucket}/area_of_interest.csv')

aoi_df.head()

In [ ]:
aoi_gdf = gpd.GeoDataFrame(
    aoi_df, 
    geometry=gpd.GeoSeries.from_wkb(aoi_df['geom'])
).drop(columns=['geom'])

aoi_gdf.head()

## 2. Visualizamos las capas en un mapa

In [ ]:
# Define the layer to display in Pydeck use expression to style by radius

layer = pdk.Layer('GeoJsonLayer', # E.g: switch to 'HeatmapLayer'
                  data=soja_gdf,
                  get_position='geometry.coordinates',
                  get_point_radius='5 + (soyb_a * 0.5)',
                  get_fill_color=[178, 34, 34],
                  pickable=True,
        )


aoi_layer = pdk.Layer('GeoJsonLayer', # E.g: switch to 'HeatmapLayer'
                  data=aoi_gdf,
                  get_position='geometry.coordinates',
                  storked=True,
                  get_fill_color=[0, 0, 0, 0],
                  get_line_color=[134, 19, 218],  
                  get_line_width=10_000,  
                  pickable=True,
        )


# Define the view state and set the initial camera position
view_state = pdk.ViewState(latitude=40.4168, longitude=-3.7038, zoom=1)

# Define the map object and display the layer
viz = pdk.Deck(
    layers=[aoi_layer, layer], 
    initial_view_state=view_state, 
    map_provider='carto', 
    map_style='light',
)

viz.to_html()

## Usamos un join espacial para filtrar los datos dentro de nuestra AOI

In [ ]:
soja_aoi = soja_gdf.sjoin(aoi_gdf, how='inner')

soja_aoi.head()

In [ ]:
# Define the layer to display in Pydeck use expression to style by radius

layer = pdk.Layer('GeoJsonLayer', # E.g: switch to 'HeatmapLayer'
                  data=soja_aoi,
                  get_position='geometry.coordinates',
                  get_point_radius='5 + (soyb_a * 0.5)',
                  get_fill_color=[178, 34, 34],
                  pickable=True,
        )



# Define the view state and set the initial camera position
view_state = pdk.ViewState(latitude=40.4168, longitude=-3.7038, zoom=1)

# Define the map object and display the layer
viz = pdk.Deck(
    layers=[layer], 
    initial_view_state=view_state, 
    map_provider='carto', 
    map_style='light',
)

viz.to_html()

## 3. Agregamos nuestros datos de cultivo de soja usando H3

In [ ]:
resolution = 6

soja_aoi[f'h3_res{resolution}'] = soja_aoi['geometry'].apply(point_to_h3, args=(resolution,))

soja_aoi.head(3)

In [ ]:
soja_h3 = soja_aoi.groupby(by=f'h3_res{resolution}')[['soyb_a']].sum().reset_index()

soja_h3.head()

In [ ]:
style_expression, legend = equal_color_intervals(
    soja_h3, 
    'soyb_a', 
    'Temps', 
    5, 
    return_legend=True, 
    legend_title='Producción de Soja (mt)'
)

display(legend)

h3_layer = pdk.Layer(
    "H3HexagonLayer",
    data=soja_h3,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=True,
    get_elevation='1 + (soyb_a * 1.5)',
    get_hexagon=f'h3_res{resolution}',
    get_fill_color=style_expression,
    get_line_color=[255, 255, 255],
    line_width_min_pixels=3,
)

view_state = pdk.ViewState(
    latitude=-14.235,
    longitude=-51.9253,
    zoom=4,
    pitch=40.5,
    bearing=-27.36
)

r = pdk.Deck(
        layers=[h3_layer], 
        initial_view_state=view_state, 
        map_provider='carto', 
        map_style='dark',
)

r.to_html()

# 4. Contextualizamos los datos con información sobre area quemada 

Usaremos el modelo de la NASA MCD64A1.006 MODIS Burned Area Monthly Global 500m product

In [ ]:
with rio.open(f'{gcs_bucket}/BurnedAreaAOI_07_10_south_america.tiff') as src:
    # Print some metadata
    print(f'Raster shape: {src.shape}')
    print(f'Number of bands: {src.count}')
    print(f'Coordinate reference system: {src.crs}')
    
    # Read the raster data as a numpy array
    raster = src
    band = src.read(1) # use 1 to read the first band

fig, ax = plt.subplots(figsize=(6, 10))    
    
plt.imshow(band, cmap='viridis')

In [ ]:
burned_h3 = raster_band_to_pandas_h3(
    band, 
    raster, 
    resolution, 
    h3_col_name=f'h3_res{resolution}'
)

burned_h3.head()

In [ ]:
soja_burned_h3 = soja_h3.merge(burned_h3, on=f'h3_res{resolution}',  how='left')

soja_burned_h3['burned'] = soja_burned_h3['value'] > 0

soja_only_burned_h3 = soja_burned_h3[soja_burned_h3['burned']]

soja_only_burned_h3.head()

In [ ]:
soja_burned_h3_layer = pdk.Layer(
    "H3HexagonLayer",
    data=soja_only_burned_h3,
    pickable=True,
    stroked=True,
    filled=True,
    opacity=0.4,
    get_hexagon=f'h3_res{resolution}',
    get_fill_color=style_expression,
    get_line_color=[255, 255, 255],
    line_width_min_pixels=2,
)

burned_h3_layer = pdk.Layer(
    "H3HexagonLayer",
    data=burned_h3[burned_h3['value'] > 0],
    pickable=True,
    stroked=True,
    filled=True,
    opacity=0.4,
    get_hexagon=f'h3_res{resolution}',
    get_fill_color=[255, 255, 255],
    get_line_color=[255, 255, 255],
    line_width_min_pixels=2,
)

view_state = pdk.ViewState(
    latitude=-12.6819,
    longitude=-55.4209,
    zoom=7,
)

r = pdk.Deck(
        layers=[burned_h3_layer, soja_burned_h3_layer], 
        initial_view_state=view_state, 
        map_provider='google_maps', 
        map_style='satellite',
)

r.to_html()

# TIP: Run it with resolution 8 for a better representation of reality